In [4]:
import numpy as np
import tensorflow as tf
from datetime import datetime
%run code/layerutils
%run code/preprocessing

In [12]:
### construction
num_training_examples = 11064
decay = 0.01
n = 1
X = tf.placeholder(tf.float32, shape=(None, 512, 512, 3), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')
training = tf.placeholder(tf.bool, name='training')
with tf.name_scope('cnn'):
    with tf.variable_scope('intro1'):
        out = conv(X, 64, ksize=7, stride=2)
        out = max_pool(out)
    with tf.variable_scope('intro2'):
        out = conv(out, 64, ksize=7, stride=2)
    out = group(out, n, 'group1', downsample=False)
    out = group(out, n, 'group2')
    out = group(out, n, 'group3')
    out = flatten(out)
    logits = fc(out, 2, 'fc')
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    base_loss = tf.reduce_mean(xentropy)
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_losses, name='loss')
learning_rate = 0.1
with tf.name_scope('train'):
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9, use_nesterov=True)
    training_op = optimizer.minimize(loss)
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epochs = 10
batch_size = 100
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)
xentropy_summary = tf.summary.scalar('xentropy', loss)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
with tf.Session() as sess:
    init.run()
    saver.restore(sess, './cnn1.ckpt')
    for epoch in range(3):
        for iteration in range(3):
            X_batch, y_batch = data(iteration, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            print(iteration, end=' ', flush=True)
            save_path = saver.save(sess, './cnn1.ckpt')
            if iteration % 10 == 0:
                summary_str = xentropy_summary.eval(feed_dict={X: X_batch, y: y_batch, training: True})
                step = epoch * (11064 // batch_size) + iteration
                file_writer.add_summary(summary_str, step)
        learning_rate = learning_rate * 1/(1 + decay * epoch)
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch, training: False})
        #acc_test = accuracy.eval(feed_dict={X: test_features, y: test_labels, training: False})
        print(epoch, 'Train accuracy:', acc_train)
    save_path = saver.save(sess, './cnn1.ckpt')

INFO:tensorflow:Restoring parameters from ./cnn1.ckpt
0 1 2 0 Train accuracy: 0.0
0 1 2 

KeyboardInterrupt: 

In [11]:
tf.reset_default_graph()